In [1]:
print("OK")

OK


In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import Ollama

In [3]:
# Read the PDF files from the folder

file_dir = "../dataset"
loader = PyPDFDirectoryLoader(file_dir)
docs = loader.load()

In [7]:
docs[11].page_content

'BUILD RESILIENT INFRASTRUCTURE, \nPROMOTE INCLUSIVE AND SUSTAINABLE \nINDUSTRIALIZATION AND FOSTER \nINNOVATION \nTechnological progress helps us address big global challenges such as \ncreating jobs and becoming more energy efficient. For example, the \nworld is becoming ever more interconnected and prosperous thanks to \nthe internet. The more connected we are, the more we can all benefit \nfrom the wisdom and contributions of people everywhere on earth.  And \nyet four billion people have no way of getting online, the vast majority \nof them in developing countries. The more we invest in innovation \nand infrastructure, the better off we’ll all be. Bridging the digital divide, \npromoting sustainable industries, and investing in scientific research and \ninnovation are all important ways to facilitate sustainable development.'

In [5]:
len(docs)

27

In [8]:
# Extract the text from the PDF
pdf_str = ""

for page in docs:
    pdf_str += page.page_content

In [10]:
pdf_str[0:25]

'IN THE YEAR 2015, LEADERS'

In [11]:
len(pdf_str)

19986

In [16]:
# Split the documents into chunks
model = "gpt-3.5-turbo"

text_splitter = TokenTextSplitter(
    model_name = model,
    chunk_size = 10000,
    chunk_overlap = 200
)
pdf_doc = text_splitter.split_text(pdf_str)

In [18]:
pdf_doc

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [19]:
len(pdf_doc)

1

In [26]:
type(pdf_doc[0])

str

In [20]:
# Convert string to langchain documents
from langchain.docstore.document import Document

doc = [Document(page_content = t) for t in pdf_doc]

In [22]:
doc[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [25]:
type(doc[0])

langchain_core.documents.base.Document

In [27]:
# Split the documents into chunks
model = "gpt-3.5-turbo"

text_splitter = TokenTextSplitter(
    model_name = model,
    chunk_size = 1000,
    chunk_overlap = 100
)
pdf_docs = text_splitter.split_documents(doc)

In [31]:
pdf_docs[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 

In [32]:
len(pdf_docs)

6